---
title: Superposition as a Super Datalog
date: 2024-12-02
---

[Resolution](https://en.wikipedia.org/wiki/Resolution_(logic)) is an old technique in automated reasoning. [Datalog](https://en.wikipedia.org/wiki/Datalog) is a family of languages capable of expressing recursive database queries. The ancestry of datalog can be traced back to resolution and it is interesting and fruitful to examine the capabilities of modern resolution style provers in light of the use cases and operational interpretability of datalog.

Datalog is quite simple. You have a starting set of facts in relations for example the edges in a graph. These can be stored as a relational table.

```prolog
edge(1,2).
edge(2,3).
```

Then you have rules that iteratively derive new facts from the currently existing facts. The is done by making a database query out of the body of the rule, and inserting the head with the found values.

```prolog
path(X,Y) :- edge(X,Y).  % if there is an edge, there is a path
path(X,Y) :- edge(X,Z), path(Z,Y). % You can paste an edge on the end of a path to get a new path.
```

A good datalog is [souffle](https://souffle-lang.github.io/simple). 


In [81]:
%%file /tmp/path.dl
.decl edge(x:number, y:number)
edge(1,2).
edge(2,3).

.decl path(x:number, y:number)
.output path(IO=stdout)
path(X,Y) :- edge(X,Y).
path(X,Y) :- edge(X,Z), path(Z,Y).

Overwriting /tmp/path.dl


In [82]:
! souffle /tmp/path.dl

---------------
path
1	2
1	3
2	3


You can also express basically the same thing to a first order theorem prover.

In [ ]:
%%file /tmp/path.p

cnf(edgeab, axiom, edge(a,b)).
cnf(edgebc, axiom, edge(b,c)).
cnf(edge_is_path , axiom, path(X,Y) | ~edge(X,Y)).
cnf(path_trans, axiom, path(X,Z) | ~edge(X,Y) | ~path(Y,Z)).

% alternative fof syntax for rules. There are more https://www.tptp.org/
%fof(edge_is_path , axiom, ![X,Y] : (edge(X,Y) => path(X,Y))).
%fof(path_transm, axiom, ![X,Y,Z] : ((path(X,Z) & edge(X,Y)) => path(Y,Z))).

Overwriting /tmp/path.p


We can see the first two assertions establish the base edge facts and the the last two establish rules. To translate the logic programming rules to a first order ATP clause, we take the classical correspondence that implication `a -> b` is equivalent to `~a | b`. Hence for example  `path(X,Z) :- edge(X,Y), path(Y,Z).` becomes `path(X,Z) | ~edge(X,Y) | ~path(Y,Z)`. We can kind of think of negation `~` as a marker stating this literal goes in the body of the rule. There is an alternate prolog universe in which this was standard notation.

Two premier resolution style theorems provers are [Vampire](https://vprover.github.io/usage.html) and [eprover](https://github.com/eprover/eprover). They are usually top in class in the [CADE ATP competition](https://tptp.org/CASC/).

In [108]:
! vampire /tmp/path.p

% Running in auto input_syntax mode. Trying TPTP
% SZS status Satisfiable for path
% # SZS output start Saturation.
cnf(u13,axiom,
    ~edge(b,X0) | path(X0,c)).

cnf(u12,axiom,
    ~edge(a,X0) | path(X0,b)).

cnf(u15,axiom,
    path(c,c)).

cnf(u17,axiom,
    ~edge(c,X0) | path(X0,c)).

cnf(u19,axiom,
    ~edge(c,X0) | path(X0,b)).

cnf(u14,axiom,
    path(b,b)).

cnf(u16,axiom,
    ~edge(b,X0) | path(X0,b)).

cnf(u9,axiom,
    ~path(X0,X2) | ~edge(X0,X1) | path(X1,X2)).

cnf(u18,axiom,
    path(c,b)).

cnf(u8,axiom,
    ~edge(X0,X1) | path(X0,X1)).

cnf(u1,axiom,
    edge(a,b)).

cnf(u11,axiom,
    path(b,c)).

cnf(u10,axiom,
    path(a,b)).

cnf(u2,axiom,
    edge(b,c)).

% # SZS output end Saturation.
% ------------------------------
% Version: Vampire 4.9 (commit 5ad494e78 on 2024-06-14 14:05:27 +0100)
% Linked with Z3 4.12.3.0 79bbbf76d0c123481c8ca05cd3a98939270074d3 z3-4.8.4-7980-g79bbbf76d
% Termination reason: Satisfiable

% Memory used [KB]: 429
% Time elapsed: 0.0000 s
perf_

Oh wow, sweet. That was easy. Blog post over.

No, sorry.

Really rule of thumb is that you should never be running either vampire or eprover without a `--mode` or `--auto` flag. The default modes are enormously slower.

In `casc_sat` mode we get piles of difficult to interpret garbage.

In [107]:
! vampire --mode casc_sat /tmp/path.p

% Running in auto input_syntax mode. Trying TPTP
% WARNING: time unlimited strategy and instruction limiting not in place - attempting to translate instructions to time
% fmb+10_1:1_sil=256000:i=98885:tgt=full:fmbsr=1.3:fmbss=10_0 on path for (495ds/98885Mi)
Detected minimum model sizes of [1]
Detected maximum model sizes of [3]
TRYING [10]
Finite Model Found!
% SZS status Satisfiable for path
perf_event_open failed (instruction limiting will be disabled): Permission denied
(If you are seeing 'Permission denied' ask your admin to run 'sudo sysctl -w kernel.perf_event_paranoid=-1' for you.)
% Solution written to "/tmp/vampire-proof-3398902"
% SZS output start FiniteModel for path
tff('declare_$i1',type,a:$i).
tff('declare_$i2',type,b:$i).
tff('declare_$i3',type,'fmb_$i_3':$i).
tff('declare_$i4',type,a:$i).
tff('declare_$i5',type,a:$i).
tff('declare_$i6',type,a:$i).
tff('declare_$i7',type,b:$i).
tff('declare_$i8',type,a:$i).
tff('declare_$i9',type,a:$i).
tff('declare_$i10',type,'fmb_$i_1

eprover in it's default mode does not terminate/saturate on this problem. We can kind of see the problem that it is resolving rules against rules, building larger and larger transitivity clauses.

In [111]:
! eprover-ho --total-clause-set-limit=15 /tmp/path.p

# Initializing proof state
# Scanning for AC axioms
#
#cnf(i_0_3, plain, (edge(a,b))).
#
#cnf(i_0_4, plain, (edge(b,c))).
#
#cnf(i_0_5, plain, (path(X1,X2)|~edge(X1,X2))).
#
#cnf(i_0_6, plain, (path(X3,X2)|~path(X1,X2)|~edge(X1,X3))).
#
#cnf(i_0_8, plain, (path(X1,X2)|~edge(X3,X1)|~edge(X3,X2))).
#
#cnf(i_0_7, plain, (path(X1,X2)|~path(X4,X2)|~edge(X3,X1)|~edge(X4,X3))).
#
#cnf(i_0_9, plain, (path(X1,X2)|~edge(X3,X1)|~edge(X4,X3)|~edge(X4,X2))).
##
#cnf(i_0_10, plain, (path(X1,X2)|~path(X4,X2)|~edge(X3,X1)|~edge(X5,X3)|~edge(X4,X5))).

# Failure: User resource limit exceeded!
# SZS status ResourceOut


However, we can get it to terminate with a reasonable database if we start tweaking the options controlling it's execution.

In [112]:
! eprover-ho --print-saturated --literal-selection-strategy="SelectNegativeLiterals" /tmp/path.p

# Initializing proof state
# Scanning for AC axioms
#
#cnf(i_0_3, plain, (edge(a,b))).
#
#cnf(i_0_4, plain, (edge(b,c))).
#
#cnf(i_0_5, plain, (path(X1,X2)|~edge(X1,X2))).
#
#cnf(i_0_7, plain, (path(b,c))).
#
#cnf(i_0_8, plain, (path(a,b))).
#
#cnf(i_0_6, plain, (path(X3,X2)|~path(X1,X2)|~edge(X1,X3))).
#
#cnf(i_0_9, plain, (path(X1,c)|~edge(b,X1))).
#
#cnf(i_0_11, plain, (path(c,c))).
#
#cnf(i_0_10, plain, (path(X1,b)|~edge(a,X1))).
#
#cnf(i_0_13, plain, (path(b,b))).
#
#cnf(i_0_12, plain, (path(X1,c)|~edge(c,X1))).
#
#cnf(i_0_14, plain, (path(X1,b)|~edge(b,X1))).
#
#cnf(i_0_15, plain, (path(c,b))).
#
#cnf(i_0_16, plain, (path(X1,b)|~edge(c,X1))).

# No proof found!
# SZS status Satisfiable
# Processed positive unit clauses:
cnf(i_0_3, plain, (edge(a,b))).
cnf(i_0_4, plain, (edge(b,c))).
cnf(i_0_7, plain, (path(b,c))).
cnf(i_0_8, plain, (path(a,b))).
cnf(i_0_11, plain, (path(c,c))).
cnf(i_0_13, plain, (path(b,b))).
cnf(i_0_15, plain, (path(c,b))).

# Processed negative unit clauses:



So these provers in their rawest black box form can't really be used as datalogs, but if we peer under the covers a bit at the operational mechanisms, we can do it.

Even more so than that, in terms of pure raw mechanism these systems subsume things like equality saturation and knuth bendix completion. The difficulty id controlling them well enough to get the thing we want out.

# Ordered Resolution and Selection

The naive form of ground binary resolution looks like this. Regardless of whether A is true or false, we know one at least one of the clauses C and D is true.

```
C \/ A    ~A \/ D
-----------------
     C \/ D  
```

Our nontermination with eprover is because yea, this would just keep growing because the rule `path(X,Z) | ~edge(X,Y) | ~path(Y,Z)` could resolve against itself forever.

[Ordered resolution](https://lawrencecpaulson.github.io/papers/bachmair-hbar-resolution.pdf) is a paradigm for restricting the resolution process while maintaining completeness. It only requires certain resolutions to go through that fit the conditions of a term ordering and literal selection function.

An idea that is both simple and complex is that of [Term Orderings](https://www.cs.unm.edu/~mccune/prover9/manual/2009-11A/term-order.html). Sometimes, it makes sense to have an ordering on terms (simpler, smaller, faster, more expanded/foiled, defined earlier are all possible natural partial orders on terms). Having this play nice with unification variables/substitution and/or theories and/or alpha renaming and/or [lambda computation](https://arxiv.org/abs/1506.03943) is quite a bit trickier (rewrite and simplification orderings). 

Ordered resolution uses a term ordering sort of as a recipe on how to orient a `|` clause into a directed `:-` rule. If the largest literal is positive ("productive clauses"), it picked as the head. Other positive literals become negations in the body of the rule. If the largest literal is negative, then the entire clause is a constraint (Kind of a rule with head false `:- a, b, c.` You see these sorts of things in answer set programming, or you could see it if you are trying to refutation prove using datalog `false() :- a(),b(),c().`).

At the same time, this ordering puts an ordering on the possible [Herbrand models](https://en.wikipedia.org/wiki/Herbrand_structure) (Herbrand models are sets of ground terms) of the theory, and it becomes possible to speak of a minimal model as we do in the context of datalog. A big distinction between SAT solvers and ATPs and Datalog/Answer Set Programming is a notion of ["justification"](https://www.philipzucker.com/minikanren_inside_z3/). Datalog only derives facts that are forced to be true. SAT solvers seemingly give you a random model if there is one. Datalog gives you a particular "best" model. There is a sense in which SAT solvers are also giving you a minimal model. It is minimal with respect to the current internal backtracking trail of the solvers, so a perspective on SAT is that it is building both a model and "good" literal ordering, or pivoting its literal ordering on the fly.

From the perspective of refutation proving, when one wants to show there can't be a satisfying model, it is sufficient to show there can't be a minimal model.

This is reminiscent of the notion of [symmetry breaking](https://en.wikipedia.org/wiki/Symmetry-breaking_constraints) in optimization that if there is some symmetry in your model (all reds can be swapped with blues in a graph coloring for example or permutations of vehicles in a routing problem), that it is useful to put an an extra ordering constraint to reduce the search space (prefer reds).

A term ordering naturally extends to an ordering on clauses.

During the datalog like process to produce a minimal model, if we hit a failing constraint, then there should have been an ordered resolution step possible to make a rule in an earlier "strata" that would have avoided the constraint failure. Carefully describing this idea is the proof of completeness of ordered resolution.

The literal selection function is basically an arbitrary function from clauses to literals. It is surprising to find that much flexibility.

Frankly, it's all a little confusing. `--literal-selection-strategy="SelectNegativeLiterals"` is sufficient to make our particular example saturate. Other possible options also work.

See slide 13 https://www.tcs.ifi.lmu.de/lehre/ws-2024-25/atp/slides07-more-resolution.pdf

![](/assets/ordered_resolve.png)



The key thing that is making our example terminate is probably "Nothing is Selected in D \/ B". The "rule" clauses have negative literlas in them, so they will have something selected by the SelectNegativeLiterals strategy. The facts are a single positive unit literal, so will never have anything selected.

# Super Datalog and Beyond
These superposition systems however go far beyond datalog. It is under the capabilities of their calculus to be (with the appropriate clause prioerity, selection functions, and literal orderings)

- Knuth Bendix completion
- Prolog
- Egraphs
- Equality Saturation
- Hypothetical Datalog https://www.philipzucker.com/contextual-datalog/
- Contextual Egraphs https://github.com/eytans/easter-egg https://arxiv.org/abs/2305.19203
- Lambda egraphs via the new HO extensions to eprover

## Call Graphs as Ordering in Prolog and Datalog

Prolog and Datalog have a couple different notions of ordering in them.

- In prolog, the predicates in the body are ordered and the order that the rules appear also matters. More than that we kind of think of prolog as sort of a chain of function calls. The rules with a particular head define that predicate, and the body of the rule are the functions they rely on. The rules are an almost direct representation of the call graph https://en.wikipedia.org/wiki/Call_graph of the program (the graph of which functions call which functions). Recursive calls show up as self edges in the call graph, or mutually recursive definitions show up as connected components.
- In datalog, there is a derived notion of strata. Strata are again the connected components of the dependency graph of predicates. They are useful to note as an optimization, but crucially semantically important to note for one proper notion of negation

Prolog like goals are encoded as negated literals.


## Given Clause and Semi Naive Evaluation
Most if not all resolution style saturating theorem provers are organized around a given clause loops.

A pile of processed and unprocessed clauses are maintained. A clause in selected out of the unprocessed (the given clause) and all inference against the processed clauses are done. The results are thrown into the unprocessed.

This is similar to the seminaive distinction of delta (unprocessed) and old (processed) relations. The given clause procedure makes sure there is always one new thing in every attempt at inference because otherwise you are just redundantly rediscovering inferences.

Both Datalog and resolution can be executed in the naive style where you do a full inference sweep of the entire database every time.

The clause selection heuristics pick which clause to pick to be the given clause. 

## Question Answering

Question answering mode in ATPs gives you prolog like capabilities to return a substitution that makes the goal true.
 
 For example, consider this definition of an `add` predicate.

In [ ]:
%%file /tmp/add.pl
add(0,X,X).
add(s(X),Y,s(Z)) :- add(X,Y,Z).

Overwriting /tmp/add.pl


In [138]:
!swipl -s /tmp/add.pl -g "add(X,Y,s(s(0))),writeln([X,Y]),fail"

[0,s(s(0))]
[s(0),s(0)]
[s(s(0)),0]
ERROR: -g add(X,Y,s(s(0))),writeln([X,Y]),fail: false


The equivalent in TPTP would be

In [ ]:
%%file /tmp/prolog.p
cnf(add_succ, axiom, add(s(X),Y,s(Z)) | ~add(X, Y, Z)).
cnf(add_z, axiom, add(z, Y, Y)).
fof(add_g, conjecture, ?[X,Y]: add(X,Y,s(s(z)))).

In [127]:
! eprover-ho --answers=3 --conjectures-are-questions /tmp/prolog.p

# Initializing proof state
# Scanning for AC axioms
#
#cnf(i_0_4, plain, (add(z,X1,X1))).
#
#cnf(i_0_3, plain, (add(s(X1),X2,s(X3))|~add(X1,X2,X3))).
#
#cnf(i_0_5, negated_conjecture, ($answer(esk1_2(X1,X2))|~add(X1,X2,s(s(z))))).
## SZS status Theorem
# SZS answers Tuple [[z, s(s(z))]|_]

#cnf(i_0_6, negated_conjecture, ($answer(esk1_2(z,s(s(z)))))).
#
#cnf(i_0_7, negated_conjecture, ($answer(esk1_2(s(X1),X2))|~add(X1,X2,s(z)))).
## SZS answers Tuple [[s(z), s(z)]|_]

#cnf(i_0_8, negated_conjecture, ($answer(esk1_2(s(z),s(z))))).
#
#cnf(i_0_9, negated_conjecture, ($answer(esk1_2(s(s(X1)),X2))|~add(X1,X2,z))).
## SZS answers Tuple [[s(s(z)), z]|_]

# Proof found!


In [173]:
! vampire --question_answering answer_literal --avatar off /tmp/prolog.p

% Running in auto input_syntax mode. Trying TPTP
% Refutation found. Thanks to Tanya!
% SZS status Theorem for prolog
% SZS answers Tuple [[z,s(s(z))]|_] for prolog
% SZS output start Proof for prolog
2. add(z,X1,X1) [input]
3. ? [X0,X1] : add(X0,X1,s(s(z))) [input]
4. ~? [X0,X1] : add(X0,X1,s(s(z))) [negated conjecture 3]
5. ~? [X0,X1] : (add(X0,X1,s(s(z))) & ans0(X0,X1)) [answer literal 4]
6. ! [X0,X1] : (~add(X0,X1,s(s(z))) | ~ans0(X0,X1)) [ennf transformation 5]
7. ~add(X0,X1,s(s(z))) | ~ans0(X0,X1) [cnf transformation 6]
8. ~ans0(z,s(s(z))) [resolution 7,2]
9. ans0(X0,X1) [answer literal]
10. $false [unit resulting resolution 9,8]
% SZS output end Proof for prolog
% ------------------------------
% Version: Vampire 4.9 (commit 5ad494e78 on 2024-06-14 14:05:27 +0100)
% Linked with Z3 4.12.3.0 79bbbf76d0c123481c8ca05cd3a98939270074d3 z3-4.8.4-7980-g79bbbf76d
% Termination reason: Refutation

% Memory used [KB]: 415
% Time elapsed: 0.0000 s
perf_event_open failed (instruction limitin

# Equational Reasoning

## Union Find

In a my egraphs 2024 talk https://www.philipzucker.com/egraph2024_talk_done/ , I showed how to use Twee to get a union find. This is the same idea using eprover

In [169]:
%%file /tmp/uf.p

cnf(ax1, axiom, a = b).
cnf(ax2, axiom, b = c).
cnf(ax2, axiom, c = b).
cnf(ax2, axiom, b = z).
cnf(ax2, axiom, z = c).
cnf(ax3, axiom, d = e).

Overwriting /tmp/uf.p


In [170]:
! eprover-ho --print-saturated /tmp/uf.p

# Initializing proof state
# Scanning for AC axioms
#
#cnf(i_0_7, plain, (b=a)).
#
#cnf(i_0_8, plain, (c=a)).
##
#cnf(i_0_10, plain, (z=a)).
##
#cnf(i_0_12, plain, (d=e)).

# No proof found!
# SZS status Satisfiable
# Processed positive unit clauses:
cnf(i_0_7, plain, (b=a)).
cnf(i_0_8, plain, (c=a)).
cnf(i_0_10, plain, (z=a)).
cnf(i_0_12, plain, (d=e)).

# Processed negative unit clauses:

# Processed non-unit clauses:

# Unprocessed positive unit clauses:

# Unprocessed negative unit clauses:

# Unprocessed non-unit clauses:




## Egraph

I also mentioned in the talk how you can use knuth bendix completion to build an egraph out of ground equations, represent by its ground ocmpleted rewrite system

![](https://www.philipzucker.com/assets/egraph2024/egraph2.svg)
![](https://www.philipzucker.com/assets/egraph2024/egraphs_1.svg)

In [147]:
%%file /tmp/groundshift.p
fof(shift, axiom, mul(a,two) = shift(a, one)).
fof(assoc, axiom, div(mul(a,two),two) = mul(a,div(two,two))).
fof(cancel, axiom, div(two,two) = one).
fof(unit_mul, axiom, mul(a,one) = a). 
fof(cancel, axiom, myterm = div(mul(a,two), two)).


Overwriting /tmp/groundshift.p


In [149]:
! eprover-ho --print-saturated --order-weights="myterm:100"  /tmp/groundshift.p

setting user weights
# Initializing proof state
# Scanning for AC axioms
#
#cnf(i_0_4, plain, (mul(a,one)=a)).
#
#cnf(i_0_3, plain, (div(two,two)=one)).
#
#cnf(i_0_5, plain, (myterm=div(mul(a,two),two))).
#
#cnf(i_0_1, plain, (shift(a,one)=mul(a,two))).
#
#cnf(i_0_2, plain, (div(mul(a,two),two)=a)).
#
#cnf(i_0_5, plain, (myterm=a)).

# No proof found!
# SZS status Satisfiable
# Processed positive unit clauses:
cnf(i_0_4, plain, (mul(a,one)=a)).
cnf(i_0_3, plain, (div(two,two)=one)).
cnf(i_0_1, plain, (shift(a,one)=mul(a,two))).
cnf(i_0_2, plain, (div(mul(a,two),two)=a)).
cnf(i_0_5, plain, (myterm=a)).

# Processed negative unit clauses:

# Processed non-unit clauses:

# Unprocessed positive unit clauses:

# Unprocessed negative unit clauses:

# Unprocessed non-unit clauses:




## Knuth Bendix Completion
Knuth Bendix completion is a way to possibly turn a system of equations into a conlfuent terminating rewrite system. One example where t5his is possible is the free group. The default ordering of E is Knuth Bendix Ordering.

https://www.metalevel.at/trs/


In [139]:
%%file /tmp/grp.p
cnf(ax1, axiom, mul(X,mul(Y,Z)) = mul(mul(X,Y),Z)).
cnf(ax2, axiom, mul(e,X) = X).
cnf(ax3, axiom, mul(inv(X), X) = e).

Overwriting /tmp/grp.p


In [140]:
!eprover-ho --print-saturated /tmp/grp.p

# Initializing proof state
# Scanning for AC axioms
# mul is associative
#
#cnf(i_0_5, plain, (mul(e,X1)=X1)).
#
#cnf(i_0_6, plain, (mul(inv(X1),X1)=e)).
#
#cnf(i_0_4, plain, (mul(mul(X1,X2),X3)=mul(X1,mul(X2,X3)))).
#
#cnf(i_0_8, plain, (mul(inv(X1),mul(X1,X2))=X2)).
#
#cnf(i_0_13, plain, (mul(inv(e),X1)=X1)).
#
#cnf(i_0_12, plain, (mul(inv(inv(X1)),e)=X1)).
#
#cnf(i_0_16, plain, (mul(inv(inv(e)),X1)=X1)).
#
#cnf(i_0_23, plain, (inv(e)=e)).
#
#cnf(i_0_11, plain, (mul(inv(inv(X1)),X2)=mul(X1,X2))).
#
#cnf(i_0_12, plain, (mul(X1,e)=X1)).
#
#cnf(i_0_33, plain, (inv(inv(X1))=X1)).
#
#cnf(i_0_38, plain, (mul(X1,inv(X1))=e)).
#
#cnf(i_0_39, plain, (mul(X1,mul(inv(X1),X2))=X2)).
##
#cnf(i_0_42, plain, (mul(X1,mul(X2,inv(mul(X1,X2))))=e)).
#
#cnf(i_0_61, plain, (mul(X2,inv(mul(X1,X2)))=inv(X1))).
#
#cnf(i_0_78, plain, (mul(inv(mul(X1,X2)),X1)=inv(X2))).
#
#cnf(i_0_77, plain, (inv(mul(X2,X1))=mul(inv(X1),inv(X2)))).
##
# No proof found!
# SZS status Satisfiable
# Processed positive unit clause

## Egglog

The essence of [egglog](https://github.com/egraphs-good/egglog) in the same way the essence of datalog is the transitivity query is the connected component compression query.

Here the compressed database holds a union find (The `=` facts, which are oriented according to the default term ordering). Notice that `path(b,b)` is deduped with respect to `path(d,d)` because they are expressing the same information.

In [123]:
%%file /tmp/path.p

cnf(edgeab, axiom, edge(a,b)).
cnf(edgebc, axiom, edge(b,c)).
cnf(edgecd, axiom, edge(c,b)).
cnf(edgedc, axiom, edge(d,c)).
cnf(edgecd, axiom, edge(c,d)).

cnf(edge_is_path , axiom, path(X,Y) | ~edge(X,Y)).
cnf(path_trans, axiom, path(X,Z) | ~edge(X,Y) | ~path(Y,Z)).
cnf(path_eq, axiom, X = Y | ~path(X,Y) | ~path(Y,X)).


Overwriting /tmp/path.p


In [124]:
! eprover-ho --print-saturated --literal-selection-strategy="SelectNegativeLiterals" /tmp/path.p

# Initializing proof state
# Scanning for AC axioms
#
#cnf(i_0_9, plain, (edge(a,b))).
#
#cnf(i_0_10, plain, (edge(b,c))).
#
#cnf(i_0_11, plain, (edge(c,b))).
#
#cnf(i_0_13, plain, (edge(c,d))).
#
#cnf(i_0_12, plain, (edge(d,c))).
#
#cnf(i_0_14, plain, (path(X1,X2)|~edge(X1,X2))).
#
#cnf(i_0_17, plain, (path(d,c))).
#
#cnf(i_0_18, plain, (path(c,d))).
#
#cnf(i_0_19, plain, (path(c,b))).
#
#cnf(i_0_20, plain, (path(b,c))).
#
#cnf(i_0_21, plain, (path(a,b))).
#
#cnf(i_0_16, plain, (X1=X2|~path(X2,X1)|~path(X1,X2))).
#
#cnf(i_0_22, plain, (d=c)).
#
#cnf(i_0_23, plain, (c=b)).
#
#cnf(i_0_22, plain, (d=b)).
#
#cnf(i_0_18, plain, (path(b,b))).
##
#cnf(i_0_12, plain, (edge(b,b))).
######
#cnf(i_0_15, plain, (path(X1,X2)|~path(X3,X2)|~edge(X1,X3))).
##
#cnf(i_0_28, plain, (path(X1,b)|~edge(X1,a))).

# No proof found!
# SZS status Satisfiable
# Processed positive unit clauses:
cnf(i_0_9, plain, (edge(a,b))).
cnf(i_0_21, plain, (path(a,b))).
cnf(i_0_23, plain, (c=b)).
cnf(i_0_22, plain, (d=b)).


In [167]:
%%file /tmp/eq.egg
(datatype node (a) (b) (c) (d))
(relation edge (node node))
(relation path (node node))
(edge (a) (b))
(edge (b) (c))
(edge (c) (d))
(edge (d) (c))
(edge (c) (b))

(rule ((edge x y)) ((path x y)))
(rule ((edge x y) (path y z)) ((path x z)))
(rule ((path x y) (path y x)) ((union x y)))

(run 10)
(print-function path 100)

Overwriting /tmp/eq.egg


In [168]:
!egglog /tmp/eq.egg

[INFO ] Declared sort node.
[INFO ] Declared function a.
[INFO ] Declared function b.
[INFO ] Declared function c.
[INFO ] Declared function d.
[INFO ] Declared function edge.
[INFO ] Declared function path.
[INFO ] Declared rule (rule ((edge x y))
          ((path x y))
             ).
[INFO ] Declared rule (rule ((edge x y)
           (path y z))
          ((path x z))
             ).
[INFO ] Declared rule (rule ((path x y)
           (path y x))
          ((union x y))
             ).
[INFO ] Ran schedule (repeat 10 (run)).
[INFO ] Report: Rule (rule ((edge x y)        (path y z))       ((path x z))          ): search 0.000s, apply 0.000s, num matches 9
    Rule (rule ((edge x y))       ((path x y))          ): search 0.000s, apply 0.000s, num matches 7
    Rule (rule ((path x y)        (path y x))       ((union x y))          ): search 0.000s, apply 0.000s, num matches 5
    Ruleset : search 0.000s, apply 0.000s, rebuild 0.000s
    
[INFO ] Printing up to 100 tuples of table path: 

# Bits and Bobbles 2
Altogether I find vampire more confusing to control when being used off label. I can kind of more or less interpret the various command line options to eprover in terms of the underlying calculus. Eprover also ships with a latex manual https://github.com/eprover/eprover/blob/master/DOC/eprover.tex which clarifies at least some things. Combining this + experimentation + Blanchette slides + Bachmair Ganzinger handbook of automated reasoning article is the only thing I've got.

I'm intrigued at the idea of using eprover as an operational mechanism rather than a solver for first order logic. A surface language which exposes more things like selection and clause ordering would be nice. I want a happy controllable predictable medium between datalog, prolog, and atp.

Prover9 exposes knobs and operational view is more emphasized

Are the orderings in datalog/prolog more like clause orderings, literal orderings, or selection. Or is this all not that related?
Goal clause prioritization "PreferGoals" `PreferGroundGoals` `

Hypthoetical datalog
`a -| b :- c.` The partially applied transitive rule is kind of a new hypthetical rule. If all variables are grounded in c, this is a what I have called hypothetical datalog. 


I'm not entirely clear that selection is persay the thing that makes eprover terminate. The option may be kicking on a subsumption mechanism, which is what is really making it saturate?

The only difference under --print-strategy is 

```
NoSelection: Perform ordinary superposition without selection.

SelectNegativeLiterals: Select all negative literals. For Horn clauses, this
implements the maximal literal positive unit strategy [Der91] previously
realized separately in E.
```

https://domino.mpi-inf.mpg.de/internet/reports.nsf/c125634c000710cec125613300585c64/19a118dff60c0790c12572ff002b586a/$FILE/MPI-I-2007-RG1-001.pdf 
Superposition for Finite Domains (Plain Text
Version) Thomas Hillenbrand Christoph Weidenbach


|  | Propositions | Equations |
|---|------|------------|
| Brute Search | Resolution     |   Paramodulation         |
| Ordered Search | Ordered Resolution | Superposition |
| ? | Knuth Bendix Completion |
| Ground | Ground Ordered Resolution | EGraph |
| Goal Driven, Imperative, stacky | Prolog | Functional Logic Programming |
|Bottom Up Ground | Datalog | Egglog |
| Unoriented | Clause `|` | Eq `=` |
| Oriented   | rule `:-` | Rewrite `->` |


The features that make these provers powerful refutation provers make them harder to interpret when using them for other purposes like saturation.

Vampire has a ton of bells and whistles and it is harder for me to figure out how to control them.
E exposes more options that make sense to me and has a smaller core calculus.

Both have mechanisms that defer to smt/sat solvers to try to use their strengths (see AVATAR and splitting). When this occurs, I am more confused on how to interpret the model they've found.

We are running our theorem proving somewhat unusually in that we are really seeking a notion of saturated database/clause set and not trying to show unsat / refutation proof.

At this point, I think I prefer the `cnf` notation better than the `fof` notation. The precendence of `fof` is so unintuitive to me (and I suspect ) that I end up putting.

Unlike in prolog `|` has no intrinsic ordering. In datalog, the ordering of `,` commonly does not have operational meaning either.

The transitivity closure query is a little tired, but it really is the essence of datalog.

One aspect of datalog I find compelling is that it mirrors my simplistic image of what logic even is. Logic is axioms in some language and inference rules that take in theorems and produce new theorems. The set of all provable states is the closure of the axioms under the rules.



Resolution theorem proving is a little more involved. It is 
You have clauses and you you can make an inference by smushing them together.

This is a method for classical first order logic because you can use skolemization to put the system into prenex normal form, where all the existential quantifiers are pushed outside the forall quantifiers via skolemization.  https://en.wikipedia.org/wiki/Skolem_normal_form


Clauses `a | c | ~d`

Resolution allows matching of rules against rules instead of only rule against facts.

https://dl.acm.org/doi/10.1145/321250.321253 A Machine-Oriented Logic Based on the Resolution Principle - Robinson 1965


https://en.wikipedia.org/wiki/Herbrand_structure
https://en.wikipedia.org/wiki/Herbrand%27s_theorem

```
 D \/ B      C \/ ~A
-------------------
   sigma[D \/ C]

```
- $\sigma = mgu(A,B)$
- ...
- Nothing is selected in `D \/ B`
- `~A` is selected or ...


In [85]:
import clingo
prog = """
edge(1,2;3,4).
path(X,Y) :- edge(X,Y).
"""
ctl = clingo.Control()
ctl.add(prog)
ctl.ground([("base",[])])
ctl.

In [86]:
%%file /tmp/path.p
edge(1,2; 2,3; 3,4).
path(X,Y) :- edge(X,Y).
path(X,Z) :- edge(X,Y), path(Y,Z).

Overwriting /tmp/path.p


In [90]:
! python3 -m clingo --text /tmp/path.p

edge(1,2).
edge(2,3).
edge(3,4).
path(1,2).
path(2,3).
path(3,4).
path(2,4).
path(1,3).
path(1,4).



other datalogs to checkout

https://inst.eecs.berkeley.edu/~cs294-260/sp24/2024-02-05-datalog
- https://s-arash.github.io/ascent/
- slog
- gpu datalog
- logica
- dusa

- https://www.philipzucker.com/notes/Logic/answer-set-programming/
- https://www.philipzucker.com/notes/Languages/datalog/








# Superposition

atomic ground superposition - a contextual union find
ground superposition

colored egraphs

brainiac

cruanes slides https://simon.cedeela.fr/assets/jetbrains_2021.pdf

blanchette slides https://www.tcs.ifi.lmu.de/lehre/ws-2024-25/atp/slides12-superposition.pdf

model is ground rewrite rules = egraph?


weidenbach draft?
SCL simple clause
cdcl and superposition

The layering of ordering resolution (set knuth bendix) + equations (term knuth bendix) reminds me of my idea of egraph modulo theory / theory union finds as layers.


In [ ]:

# Horn
class Clause():
    hyps : list[tuple[int,int]] # negative literals
    conc : tuple[int,int]



def superpose(c1 : Clause, c2 : Clause):
    if c1.conc[0] == c2.conc[0]:
        x,y = c1.conc[1], c2.conc[1]
        if x < y:
            x,y = y,x
        return Clause(sorted(set(c1.hyps + c2.hyps)), (x,y))

def neg_superpose(c1 : Clause, c2 : Clause):

def equality_res():
    # delete all hyps of the form x=x

def factor():




# Ordered Ground Resolution
### Model Building and Completeness
Model Existence.

We want to pick a particular herbrand model. There are many. Pick an ordering.

Minimal herbrand makes sense in the context of datalog / horn clauses...

refutation Completeness proofs in general involve building models from saturated clauses (?) Robinson https://web.stanford.edu/class/linguist289/robinson65.pdf

In egraphs, we rag on completeness as silly. How silly is it?

### Knuth Bendix Analogy
Ordered Resolution completes a set of clauses into rules.
The rules can convert a ground database into a normal implied form. (The intensional to extensional database. The base facts to the ).

Consistency checks `:- a,b,c.` may mean that it is insistent to add certain base facts.



Alternative usage.

set rewriting = datalog
ordered resolution = knuth bendix for set rewriting
rewriting (ground) herbrand models.

Take a set of extra ground facts and close them out to minimal model that contains them, or possibly say they are inconstiant.

certain ground facts are equivalent under the axioms. ordered resolution builds the decision procedure.

strata ordering
accumulating semantics. obviously confluent if no negation.
{a, c} not b --> {d}

Convert clauses into datalog rules

datalog rules are like rewrite rules. They are oriented clauses.
You can run a base set of facts into a caonnical model.
Confluence.


dually, resolution/superposition is perhaps a generalized e-unification. Negated `p(A) != p(B)` can factor into stuff.
maybe splitting makes this correspondance better?

https://www.philipzucker.com/string_knuth/


body of rules and db are made of the same thing (?) This was part of the SQL = homomorphisms post. That the query and db are surprisingly symmetrical.

| Prop  |  Eq |
|---|---|
| Unoriented | Clause    | Equation |
| Oriented   | Rule `:-` | Rewrite |
| |   Ordered Resolution | KB |




In [ ]:
def subseq(body,  db):
    pass

def replace(db, body, head):
    """replace in quotes since monotonic"""
    pass

def rewrite(db, rules):
    # aka saturate

def overlaps(body1, body2):

def deduce():
    """find all critical pairs"""
    pass

def KB(clauses): # clauses ~ E
    done = False
    


### SAT solving comparison
sat solvers do not take it an a priori literal ordering. They discover a useful one via heuristics and pivoting.



https://types.pl/@sandmouth/113528558531822923
"""
there’s a fascinating viewpoint I’ve only become aware of in the last week that sat solvers in a sense do output a minimal model, they just don’t accept a definition of minimality in the form of a literal ordering nor do they output the ordering they discovered (which exists in the form of their internal decision and propagation trail). Ordered resolution by constrast https://lawrencecpaulson.github.io/papers/bachmair-hbar-resolution.pdf takes in an a priori definition of which literals you prefer to be true/false, which in turn tells you how to orient clauses into rules. The maximum positive literal of the clause is the head and any other positive literals become negations in the rule body. It’s a really interesting knob to travel between prolog and datalog and in between. It’s kind of like user defined datalog strata or something. I think this stuff is under explained nor properly translated to a logic programming viewpoint (at least in any reference I’ve read so far)
"""

ASP maybe is like SAT except the rule orientations are chosen.
What is conflict vs producing is preordianned by user.

### Splitting
Add an extra dimension to the reolsution style search. Instead of being purely sturating, we can fork our clause databases making a guess as to which part of the clause is true.

A clause can always be split by introducing a fresh symbol `q` that hodls the shared variables. If no shared varaibles, then q is propsitional and amenable to SAT solving / branching more easily. `q` can also be kind of seen as a propsotional clause abstraction / cegar. SAT modulo Resolution.
`a(X,Y) | b(Y,W)` ->  `q(Y) | a(X,Y)`  `~q(Y) | b(Y,W)` 


Grounding. You can randomly (?) pick any instantiaions of clauses and then toss into SMT/SAT solver. If that grounding is unsat, cool.

AVATAR

https://www.cl.cam.ac.uk/~lp15/papers/Arith/case-splitting.pdf

In [ ]:
%%file 





### Logic Programming
The combination of the ordering and selection feels like a knob to tweak between prolog and datalog and other things. It would be nice to have a crisper picture of this.

- hyperresolution
- locking resolution https://www.doc.ic.ac.uk/%7Ekb/MACTHINGS/SLIDES/2013Notes/7LControl4up13.pdf boyer 1973 https://www.cs.utexas.edu/~boyer/boyer-dissertation.pdf
- unit resulting resolution

ordered resolution as a logic programming language


negative literals are goals. The Factoring rule is prolog style unification.


```python
# idea. preprocessor to convert to clauses + ordering + selection function.
# Or write own ordered resolution

import lark
grammar = """
rule: head ":-" selected "|" unselected.
"""
```

Marking particular predicates as selected or unselected.
```
:- constraint foo/2.  % means it isn't selected?
```

Or make it work like LPO. 
```
decl foo : bar. % means it is a constructor and therefore at the bottom precedence.
```

Interactively saturate with every new definition? That would be a termination check (?)
It's kind of like elf or the termination arugments given to dafny / coq / lean etc.


`head  -| ctx  :- body.` hypothetical datalog. Maybe a notation for
`maxposliteral |  selected   | other`  CHR kind of has a multipart thing. I think in principle its just multiset rewrite rules. Its a conveience to not depelte and reinsert.
It is then the theorem provers job to discharge a term ordering that makes the max pos literal, selected literals work.
selected literals can emulate hyperresolution.

prolog vs E https://wwwlehre.dhbw-stuttgart.de/~sschulz/E/FAQ.html

https://web4.ensiie.fr/~guillaume.burel/download/LFAT.pdf focusing and atp. Focusing is a mechanism for describing the imperative nature of prolog or how the sequent claculus is like an imperative machine. https://drops.dagstuhl.de/storage/00lipics/lipics-vol117-mfcs2018/LIPIcs.MFCS.2018.9/LIPIcs.MFCS.2018.9.pdf


Termination checking of logic programs.
https://www.sciencedirect.com/science/article/pii/0743106692900455  Proving termination properties of prolog programs: A semantic approach
https://www.metalevel.at/prolog/termination 
https://www.cs.unipr.it/cTI/  cTI: A Termination Inference Engine
https://github.com/atp-lptp/automated-theorem-proving-for-prolog-verification

elf had termination checking. But like simple syntactic.

IC3 and CHC.
IC3 vs saturation?
#### programming resolution
look at datalog program

continuation passing as an evaluation order forciong mechanism
https://www.swi-prolog.org/pldoc/man?section=delcont deliimited continuations?

```
call(edge(X,Y)) | ~edge(X,Y).

```

Consider more metagames I tried to play in egglog or datalog. They should be more doable.

first class clauses

`clause( poses, negs )`.

`assert`

It would be kind of cool to have computational proedciates. writeln
What was the deal with e-lop?

dcgs? Proof recording.
```
cnf(ax1, axiom, prove( A > B, Pf-Tl, Pf)).
```

This idea of staging resolution is like staging prolog.
Prolog macros? Kind of ugly as hell.
modules? Lambda prolog modules?

twelf


### Ordering

Ordering = datalog strata presciprtion

Using other term orderings -> datalog termination proofs
Or datalog consistency? Having negation but showing you won't derive then underive something.
"dynamic" stratification
This doesn't have a unique minimal model
```
b :- not a.
a :- not b.
```

This does. Stratification can't cover it though. So how to prove? Term ordering maybe.
```
b(n) :- not a(n).
a(n + 1) :- not b(n), n < 10.
```
d(1, N)
d(2, N) etc

There is some kind of redundancy symmettry in brute resolution and people have been fighting it from the beginning basically. Rewynolds, slagle, kowalski

https://www.cs.upc.edu/~%7B%7Dalbert/cpo.zip https://arxiv.org/abs/1506.03943 computbility path ordering. compare types first
LPO RPO
KBO


### Selection Functions

SL SLD resolution. They use the term selection function.
https://www.sciencedirect.com/science/article/abs/pii/0004370271900129 Linear resolution with selection function kowalski kuehner
https://en.wikipedia.org/wiki/SLD_resolution SLD resolution is so named S is for selection


Selection functions. 
p :- not p, not q, z | a,b,c.
Some kind of CLP like syntax. 
Maximalnegative/minimal does perhaps seem the most prolog like.  

Selection is like a goal ordering. a prolog rule p :- q,r,s. tries to deal with q, r, s in syntactic order. q would be selected.
But if prolog had more delcarative semantics, there might be some way for the system to pick from q,r,s as the next direction work on.


### Saturation and Proof by Consistency
inductionless induction


If we have a termination criteria / strata criteria for our prolog / datalog program, can we convert that to selection functions and a term ordering.

The prcendence gnerating routines of E seem like a nice way to fill in gaps. Syntax for constraints that put precedence or weight constraints.

Inductionless induction could be used to prove negations at lower levels?
The similaity of using induction to prove consistent (has model) is mentioned in paramodulation chapter of handbook


Could I use this to do progress and preservation proofs?
boolexpr progress proof?

```


```

Termination of lambda calculus requires the typing relation. Step-indexing. Logical relations.

### Sequents
Resolution doesn't seem to really be about classical first order logic.
neg and positive literals can be interpreted as parts of sequent. Resolution is cut.
Bachmair and ganzinger vaguely allude to macro proof steps


### Chaining

"bi-rweriting a term rewriting techniqyue for monotonic order relaTIONS" levy and agusty https://www.iiia.csic.es/~levy/papers/RTA93.pdf
https://pdf.sciencedirectassets.com/272990/1-s2.0-S1571066100X00631/1-s2.0-S1571066104002968/main.pdf Knuth-Bendix Completion for Non-Symmetric
Transitive Relations struith
https://opus.bibliothek.uni-augsburg.de/opus4/frontdoor/index/index/year/2006/docId/229 Termination of ground non-symmetric Knuth-Bendix completion

chaining slagle 72
infinitary depth first search... ?
two rewrite relations. < + termorder and < + opptermorder

ordered chaining for total orderings https://link.springer.com/chapter/10.1007/3-540-58156-1_32

Rewrite Techniques for Transitive Relations



## Misc



https://people.mpi-inf.mpg.de/~mfleury/paper/Weidenback_Book_CDCL.pdf
Weidenbach chapter 2 draft

https://core.ac.uk/download/303691264.pdf  Formalizing the metatheory of logical calculi and automatic provers in Isabelle/HOL
(invited talk)
Blanchette, Jasmin Christian

https://www.tcs.ifi.lmu.de/lehre/ws-2024-25/atp/paper.pdf  Automated Theorem Proving∗ Dr. Uwe Waldmann




Types are dsichagred differently. see weidnebach chapter.
Type check ~ static. 
A stratification of inference.
Stratification either comes from orderings, or explicit quasiquote thingy. Like z3py meta. Like two level type kovacs. like metaocaml. like HOL quote carette and farmer https://arxiv.org/abs/1802.00405
One could implement a quote mechanism inside of an ATP. Kind of an interesting idea.



Saturated clauses -> minimal model (according to that ordering)
Alterantive - push pop search in order of ordering to SAT solver.
Can cut out branches that are unsat.








Bachmair and ganzinger in handbook
https://www.isa-afp.org/entries/Functional_Ordered_Resolution_Prover.html

hypothetical datalog?
 |- 
Ground database


minimal proofs


https://rg1-teaching.mpi-inf.mpg.de/autrea-ws21/notes-3d.pdf ordered resolution with selection waldemann.

https://www.tcs.ifi.lmu.de/lehre/ws-2024-25/atp_de.html
https://www.tcs.ifi.lmu.de/lehre/ws-2024-25/atp/slides05-resolution.pdf
ordering stratifies clause set by their maximal atom

polynomial unification. Restrict unfication to do small terms first
(guassian pivoting?)


https://dl.acm.org/doi/pdf/10.1145/321420.321428 slagle 1967

Building a model from a saturation.

switching from ground to nonground makes > become !<=

sequent calculus and resolution. Cut or macro rules?
The multiset chjaracter of alsues makes permutation symettry


stratified resolution


2-sat ~ congruence closure?

`a | b` orients to `a :- b`

a | b | c. is a 3 way symmettric thing.

Multiequations a = b = c ? Could that be cogent?
a = b = c as a triangle. higher dimesional rewriting. ab -> c oriented.
Hmm. kind of jives with the equations = paths stuff.
unification sometimes uses the term.
a = b | c = d

Bachmair
Ganzigner
Niewenhuis https://www.cs.upc.edu/~roberto/
Rubio https://www.cs.upc.edu/~albert/ https://link.springer.com/chapter/10.1007/978-3-642-31585-5_21 nominal completion
waldemann https://dblp.org/pid/w/UweWaldmann.html
weidenbach
stickel
wayne snyder
hillenbrand
kuehner
kowalski
slagle
veroff
wos
suda
loveland
blanchette
bledsoe
gallier
baader
nipkow





https://resources.mpi-inf.mpg.de/SATURATE/Saturate.html
https://resources.mpi-inf.mpg.de/SATURATE/doc/Saturate/node11.html
transitive relations have special inference rule. chaining. paramdodulation is an instance...

Loveland book

https://link.springer.com/article/10.1007/BF01190829 1994 ganziner bachmair waldeman

whoa. this is insane https://people.mpi-inf.mpg.de/alumni/ag2/2011/hg/index/index7.html
I need to read like ganzinger's entire work
especially part 2.

mcallester. steensgard analysis

https://www3.risc.jku.at/conferences/rta2008/slides/Slides_Hillenbrand.pdf hillenbrand waldmeisetrt. in mathematica?

lock resolution
"the inverse method can also be encoded"

Bachmair and Ganzinger
"non-clausal reoslution and superposition wioth seelction and redunancy criteri"
"perfoect model semantics for logic programs with equality"
"rewirte based equational theorem proving with selection and simplification"
"rewrite techniques with transitive relations"
"ordered chaining for total orderings"



https://www.sciencedirect.com/science/article/pii/S0890540106000617?via%3Dihub  Modular proof systems for partial functions with Evans equality. Total and partial functions
Kuper  https://www.sciencedirect.com/science/article/pii/S0890540183710631  An Axiomatic Theory for Partial Functions
https://www21.in.tum.de/students/set_theory_partial_functions/index.html Formalising Set Theory Based on Partial Functions
https://page.mi.fu-berlin.de/cbenzmueller/papers/C57.pdf  Automating Free Logic in Isabelle/HOL

Man a return to form of category theory blog post 0. Flat is good anyway.
Avoiding junk elements is good.
Fun,El,El choiuce is like SEAR
Fun,Fun,Fun choice is like ETCS

subsets are partial functions to bool.
f(x) undef means x isn't in domain.
f(x) = false means not in subset
f(x) = true means in subset


```
DeclareSort("Fun")
# apply as ternary
apply = Function("apply", Fun, El, El, BoolSort()) # Fun Fun Fun?
undef = define("undef", [f,x], Not(Exists([y], apply(f,x,y))))
# f.x ~ y   notation for apply(f,x,y) is nice. Hard to see how to do this in python. Could make parser. Or overload __eq__ to check for a call. Hmmmm.

""" put partial application at metalevel."""

class PApply():
    f : Fun
    x : PApply | Fun # enable recursive expressions.
    def __eq__(self, y):
        if isinstance(y, ExprRef):
            return apply(self.f, self.x, y)
    def __call__(self, y):
        return PApply(self, y)
    def defined(self):
        return Exists([y], self == y)

def apply_eq(fx,y):
    if is_app(fx) and  fx.decl() == apply:
        return return fx[2] == y
    else:
    return apply(fx[0],fx[1],y)


```


https://github.com/NikolajBjorner/ShonanArtOfSAT/blob/main/AkihisaYamada-slides.pdf 
satisfiability modulo rewriting
WP
CPO
HORPO



books 
troesltra
pohlers proof thoery
takeuti
zach proof theory
handbook of proof theory
girard

kleene metamartrhemtics
computability theory by 


theory resolution stickel 1985

vampire for QBF?
```
cnf( v(B) | v(c) )
v(skolem(A))
```

vampire for modal / intuitonsitc? Judicious choice of ordering /precdence to help?
https://ieeexplore.ieee.org/document/848641 Chaining techniques for automated theorem proving in many-valued logics


nonclausal resolution
https://www.sciencedirect.com/science/article/pii/S0890540105000258 Superposition with equivalence reasoning and delayed clause normal form transformation

### Saturate
https://resources.mpi-inf.mpg.de/SATURATE/Saturate.html
INteresting system. Interesting example files.
I doubt I can get this running

Saturation of first-order (constrained) clauses with the Saturate system https://dl.acm.org/doi/10.5555/647193.720661


In [ ]:
# clauses as sorted list
# use integer ids. negative for negative literals if we want those to come first?
atoms = []






## python
We pick the (name, neg) encoding because in python 
{A,A} is neg lit
{A} is pos lit

is the same ordering as

(A, True) is neg lit
(A, False) is pos lit


Lesser things are "simpler" in some sense. "Smaller". We try to eliminate to lesser things.

Given saturated clause set, contruct model.

In [27]:
def c(*ls): return sorted(ls,reverse=True, key=lambda x: (x[0], x[1]))
def l(atom, neg=False): return (atom,neg)
def cset(*cs): return sorted(cs)
def maxlit(c): return c[0]
def maxatom(c): return c[0][0]

b0,a0,b1,a1,b2,a2 = map(lambda x: l(x), "0b 0a 1b 1a 2b 2a".split())
nb0, na0, nb1, na1, nb2, na2 = map(lambda x: l(x, neg=True),  "0b 0a 1b 1a 2b 2a".split())

N = cset(
    c(b0, a1),
    c(a0, b0),
    c(nb0, a1),
    c(nb0, b1, nb0, b1),
    c(nb0, a2, b1),
    c(nb0, na2, b1),
    c(nb1, b2),
)
N
#def Ic(n): return sum(epsC(n) for i in range(n))
#def epsC(n): return {maximal(N[n])} if  else set()




[[('0b', False), ('0a', False)],
 [('1a', False), ('0b', False)],
 [('1a', False), ('0b', True)],
 [('1b', False), ('1b', False), ('0b', True), ('0b', True)],
 [('2a', False), ('1b', False), ('0b', True)],
 [('2a', True), ('1b', False), ('0b', True)],
 [('2b', False), ('1b', True)]]

In [34]:
def interp(model, clause): 
    for atom, neg in clause:
        if neg:
            if atom not in model:
                return True
        else:
            if atom in model:
                return True
    return False
    #return any(atom not in model if neg else atom in model for (atom, neg) in model)
Ic = set()
for c in N:
    print(Ic, c, interp(Ic, c))
    if not interp(Ic, c):
        atom,neg = maxlit(c)
        if neg:
            raise Exception("counterexample", c, Ic)
        Ic.add(atom)
Ic


set() [('0b', False), ('0a', False)] False
{'0b'} [('1a', False), ('0b', False)] True
{'0b'} [('1a', False), ('0b', True)] False
{'1a', '0b'} [('1b', False), ('1b', False), ('0b', True), ('0b', True)] False
{'1a', '0b', '1b'} [('2a', False), ('1b', False), ('0b', True)] True
{'1a', '0b', '1b'} [('2a', True), ('1b', False), ('0b', True)] True
{'1a', '0b', '1b'} [('2b', False), ('1b', True)] False


{'0b', '1a', '1b', '2b'}

Selection functions take in clause and select some subset of the negative literals.

Negative and positive literals are not treated symmetrically by the model generation process.
A little odd. I think we could have the model generation have everything true by default and derive not trues

In [ ]:
def allsel(c): return [atom for (atom, neg) in c if neg]
def nonesel(c): return []
def maxsel(c): return max(allsel(c))
def minsel(c): return min(allset(c))

def ordered_resolve(pc,nc): # positive and negative clause
    (a,neg) = maxlit(pc)
    assert not neg
    (b,neg) = maxlit(nc)
    assert neg
    assert a == b
    return cset(pc[1:] + nc[1:])

# https://www.tcs.ifi.lmu.de/lehre/ws-2024-25/atp/slides07-more-resolution.pdf
def resolve_osel(pc,nc,sel):
    assert len(sel(pc)) == 0
    a,neg = maxlit(pc)
    assert not neg
    bs = sel(nc)
    if len(bs) == 0:
        b,neg = maxlit(nc)
        assert neg
        return cset(pc[1:] + nc[1:])
    else:
        for b in bs:
            if b == a:
                c = nc.copy().remove((b,True))
                c.extend(pc[1:])
                return cset(c)

def factor(c,sel):
    a,neg = maxlit(c)
    assert c[1] == (a, neg)
    assert len(sel(c) == 0)
    return c[2:]






Are the any subsumption/redundant rules that work on ordered ground resolution?

Connection to SAT search (counterexample clauses? unit propagate )
Connection to sequents and other logics. Mega steps of inference

abstract dpll
https://gist.github.com/kmicinski/17dffc8b2cbbd4f3264071e19ae75dfa See this paper: https://homepage.cs.uiowa.edu/~tinelli/papers/NieOT-JACM-06.pdf

Otter vs discount loop


In [ ]:
from collections import defaultdict
def clausedict(cs):
    pcs,ncs = defaultdict(list),defaultdict(list)
    for c in cs:
        a,neg = maxlit(c)
        if neg:
            ncs[a].append(c)
        else:
            pcs[a].append(c)
    return pcs,ncs

def saturate(N):
    passive = N
    active = []
    while passive:
        passive.pop()


A term ordering gives a clause ordering.

Producing clauses.
A tern ordering turns a set of clauses into a model generating datalog / prolog program
The produced maximal atom is the head of a clause. 
The strata are the levels.

This is ASP like.

blanchette had those slides about completeness discussing how models get built


## 2sat


# METIS
https://www.gilith.com/metis/
ordered paramodulation
https://github.com/gilith/metis

# Eprover
```
  --expert-heuristic=<arg>
    Select one of the clause selection heuristics. Currently at least
    available: Auto, Weight, StandardWeight, RWeight, FIFO, LIFO, Uniq,
    UseWatchlist. For a full list check HEURISTICS/che_proofcontrol.c. Auto
    is recommended if you only want to find a proof. It is special in that it
    will also set some additional options. To have optimal performance, you
    also should specify -tAuto to select a good term ordering. LIFO is unfair
    and will make the prover incomplete. Uniq is used internally and is not
    very useful in most cases. You can define more heuristics using the
    option -H (see below).

      --split-clauses[=<arg>]
    Determine which clauses should be subject to splitting. The argument is
    the binary 'OR' of values for the desired classes:
         1:  Horn clauses
         2:  Non-Horn clauses
         4:  Negative clauses
         8:  Positive clauses
        16:  Clauses with both positive and negative literals
    Each set bit adds that class to the set of clauses which will be split.
    The option without the optional argument is equivalent to
    --split-clauses=7.

  --split-method=<arg>
    Determine how to treat ground literals in splitting. The argument is
    either '0' to denote no splitting of ground literals (they are all
    assigned to the first split clause produced), '1' to denote that all
    ground literals should form a single new clause, or '2', in which case
    ground literals are treated as usual and are all split off into
    individual clauses.

       -H <arg>
  --define-heuristic=<arg>
    Define a clause selection heuristic (see manual for details). Later
    definitions override previous definitions.
```

Special clause type `watchlist`

## Union find

In a my egraphs 2024 talk https://www.philipzucker.com/egraph2024_talk_done/ , I showed how to use twee to get a union find. This is the same idea

In [118]:
%%file /tmp/uf.p

cnf(ax1, axiom, a = b).
cnf(ax2, axiom, b = c).
cnf(ax2, axiom, c = b).
cnf(ax2, axiom, b = z).
cnf(ax2, axiom, z = c).
cnf(ax3, axiom, d = e).
%cnf(ax4, axiom, d != a).

Overwriting /tmp/uf.p


In [119]:
! eprover-ho --print-saturated /tmp/uf.p

# Initializing proof state
# Scanning for AC axioms
#
#cnf(i_0_7, plain, (b=a)).
#
#cnf(i_0_8, plain, (c=a)).
##
#cnf(i_0_10, plain, (z=a)).
##
#cnf(i_0_12, plain, (d=e)).

# No proof found!
# SZS status Satisfiable
# Processed positive unit clauses:
cnf(i_0_7, plain, (b=a)).
cnf(i_0_8, plain, (c=a)).
cnf(i_0_10, plain, (z=a)).
cnf(i_0_12, plain, (d=e)).

# Processed negative unit clauses:

# Processed non-unit clauses:

# Unprocessed positive unit clauses:

# Unprocessed negative unit clauses:

# Unprocessed non-unit clauses:




--print-saturated=teigEIGaA --print-sat-info 

In [29]:
! eprover-ho --order-precedence-generation=none

eprover: Wrong argument to option -G (--order-precedence-generation). Possible values: none, unary_first, unary_freq, arity, invarity, const_max, const_min, freq, invfreq, invconjfreq, invfreqconjmax, invfreqconjmin, invfreqconstmin, invfreqhack, typefreq, invtypefreq, combfreq, invcombfreq, arrayopt, orient_axioms



"invfreq: Sort symbols by frequency (frequently occurring symbols are smaller).
In our experience, this is one of the best general-purpose precedence gen-
eration schemes."

"The option --literal-comparison=<arg> allow the user to select alterna-
tive literal comparison schemes. In particular, literals will be first compared by
predicate symbol, and only then by full terms. This is a poor man’s version of
transfinite KBO [LW07, KMV11], applied to literals only, but also extended to
LPO."

Ah, if I use `>` unquoted bash thinks its a file redirect

"There are two uses for a watchlist: To guide the proof search (using a heuris-
tic that prefers clauses on the watchlist), or to find purely constructive proofs
for clauses on the watchlist."

In [34]:
! eprover-ho --output-level=4  --print-saturated --term-ordering=KBO6 --precedence="a>b>c>d>e>z" /tmp/uf.p

cnf(ax1, axiom, (a=b), file('/tmp/uf.p', ax1)).
cnf(ax2, axiom, (b=c), file('/tmp/uf.p', ax2)).
cnf(ax2, axiom, (z=c), file('/tmp/uf.p', ax2)).
cnf(ax3, axiom, (d=e), file('/tmp/uf.p', ax3)).
# Initializing proof state
# Scanning for AC axioms
cnf(c_0_5, plain, (a=c),inference(rw, [status(thm)],[c_0_-9223372036854775799,c_0_-9223372036854775798])).
cnf(c_0_6, plain, (b=z),inference(rw, [status(thm)],[c_0_-9223372036854775798,c_0_-9223372036854775797])).
cnf(c_0_7, plain, (a=z),inference(rw, [status(thm)],[c_0_5,c_0_-9223372036854775797])).
cnf(c_0_8, plain, (c=z), c_0_-9223372036854775797,['final']).
cnf(c_0_9, plain, (d=e), c_0_-9223372036854775796,['final']).
cnf(c_0_10, plain, (a=z), c_0_7,['final']).
cnf(c_0_11, plain, (b=z), c_0_6,['final']).

# No proof found!
# SZS status Satisfiable
# Processed positive unit clauses:
cnf(c_0_8, plain, (c=z)).
cnf(c_0_9, plain, (d=e)).
cnf(c_0_10, plain, (a=z)).
cnf(c_0_11, plain, (b=z)).

# Processed negative unit clauses:

# Processed non-unit

In [25]:
! eprover-ho --output-level=4  --print-saturated --term-ordering=KBO6 --order-weights=a:9,b:8,c:7,d:6,e:5,z:4 /tmp/uf.p

cnf(ax1, axiom, (a=b), file('/tmp/uf.p', ax1)).
cnf(ax2, axiom, (b=c), file('/tmp/uf.p', ax2)).
cnf(ax2, axiom, (z=c), file('/tmp/uf.p', ax2)).
cnf(ax3, axiom, (d=e), file('/tmp/uf.p', ax3)).
setting user weights
# Initializing proof state
# Scanning for AC axioms
cnf(c_0_5, plain, (a=c),inference(rw, [status(thm)],[c_0_-9223372036854775799,c_0_-9223372036854775798])).
cnf(c_0_6, plain, (b=z),inference(rw, [status(thm)],[c_0_-9223372036854775798,c_0_-9223372036854775797])).
cnf(c_0_7, plain, (a=z),inference(rw, [status(thm)],[c_0_5,c_0_-9223372036854775797])).
cnf(c_0_8, plain, (c=z), c_0_-9223372036854775797,['final']).
cnf(c_0_9, plain, (d=e), c_0_-9223372036854775796,['final']).
cnf(c_0_10, plain, (a=z), c_0_7,['final']).
cnf(c_0_11, plain, (b=z), c_0_6,['final']).

# No proof found!
# SZS status Satisfiable
# Processed positive unit clauses:
cnf(c_0_8, plain, (c=z)).
cnf(c_0_9, plain, (d=e)).
cnf(c_0_10, plain, (a=z)).
cnf(c_0_11, plain, (b=z)).

# Processed negative unit clauses:


## edge path

In [16]:
%%file /tmp/path.p

cnf(ax2, axiom, path(X,Y) | ~edge(X,Y)).
cnf(ax1, axiom, path(X,Z) | ~edge(X,Y) | ~path(Y,Z)).
cnf(ax1, axiom, edge(a,b)).
cnf(ax2, axiom, edge(b,c)).
cnf(ax3, axiom, edge(c,d)).


Overwriting /tmp/path.p


NoSelection - doesm't terminate
NoGeneration - not sure what this one is doing
SelectNegativeLiterals
SelectLargestNegLit
--precedence="path>edge" 
-term-ordering=LPO4

--literal-selection-strategy="SelectSmallestNegLit" 

--auto doesn't process the path
No selection doesn't terminate.

--print-strategy 

Oh yea. Those non ground are necessary if there is no multi-resolution rule.

precedence does not seem to matter to final result

--filter-saturated='u'
--literal-selection-strategy="SelectSmallestNegLit"

In [38]:
! eprover-ho --literal-selection-strategy=none

eprover: Wrong argument to option -W (--literal-selection-strategy). Possible values: NoSelection, NoGeneration, SelectNegativeLiterals, PSelectNegativeLiterals, SelectPureVarNegLiterals, PSelectPureVarNegLiterals, SelectLargestNegLit, PSelectLargestNegLit, SelectSmallestNegLit, PSelectSmallestNegLit, SelectLargestOrientable, PSelectLargestOrientable, MSelectLargestOrientable, SelectSmallestOrientable, PSelectSmallestOrientable, MSelectSmallestOrientable, SelectDiffNegLit, PSelectDiffNegLit, SelectGroundNegLit, PSelectGroundNegLit, SelectOptimalLit, PSelectOptimalLit, SelectMinOptimalLit, PSelectMinOptimalLit, SelectMinOptimalNoTypePred, PSelectMinOptimalNoTypePred, SelectMinOptimalNoXTypePred, PSelectMinOptimalNoXTypePred, SelectMinOptimalNoRXTypePred, PSelectMinOptimalNoRXTypePred, SelectCondOptimalLit, PSelectCondOptimalLit, SelectAllCondOptimalLit, PSelectAllCondOptimalLit, SelectOptimalRestrDepth2, PSelectOptimalRestrDepth2, SelectOptimalRestrPDepth2, PSelectOptimalRestrPDepth2, S

In [26]:
! eprover-ho --print-saturated --output-level=10 --term-ordering=LPO4 --expert-heuristic=FIFO --literal-selection-strategy="SelectLargestNegLit" --precedence="path>edge"   /tmp/path.p

cnf(ax2, axiom, (path(X1,X2)|~edge(X1,X2)), file('/tmp/path.p', ax2)).
cnf(ax1, axiom, (path(X1,X2)|~edge(X1,X3)|~path(X3,X2)), file('/tmp/path.p', ax1)).
cnf(ax1, axiom, (edge(a,b)), file('/tmp/path.p', ax1)).
cnf(ax2, axiom, (edge(b,c)), file('/tmp/path.p', ax2)).
cnf(ax3, axiom, (edge(c,d)), file('/tmp/path.p', ax3)).
cnf(c_0_6, plain, (path(X1,X2)|~edge(X1,X2)),inference(fof_simplification, [status(thm)],[c_0_1])).
cnf(c_0_7, plain, (path(X1,X2)|~edge(X1,X3)|~path(X3,X2)),inference(fof_simplification, [status(thm)],[c_0_2])).
# Initializing proof state
cnf(c_0_8, plain, (edge(a,b)), c_0_-9223372036854775793,['eval']).
cnf(c_0_9, plain, (edge(b,c)), c_0_-9223372036854775792,['eval']).
cnf(c_0_10, plain, (edge(c,d)), c_0_-9223372036854775791,['eval']).
cnf(c_0_11, plain, (path(X1,X2)|~edge(X1,X2)), c_0_-9223372036854775795,['eval']).
cnf(c_0_12, plain, (path(X1,X2)|~path(X3,X2)|~edge(X1,X3)), c_0_-9223372036854775794,['eval']).
# Scanning for AC axioms
cnf(c_0_13, plain, (edge(a,b)),

## Knuth Bendix
UEQ 
http://cl-informatik.uibk.ac.at/software/mkbtt/download.php 101 problems

https://www.metalevel.at/trs/

In [33]:
%%file /tmp/grp.p
cnf(ax1, axiom, mul(X,mul(Y,Z)) = mul(mul(X,Y),Z)).
cnf(ax2, axiom, mul(e,X) = X).
cnf(ax3, axiom, mul(inv(X), X) = e).


Overwriting /tmp/grp.p


In [ ]:
! eprover-ho --print-saturated --output-level=10 --term-ordering=KBO6 --literal-selection-strategy="SelectLargestNegLit"  /tmp/grp.p

## Unification
Raw usage. Equality resolution rule ought to put unifier into ans predicate.

`er` is equality resolution. Interesting.
I needed to give it a selection to make it do this

Ah, side condition says u != v has to be eligibile for resolution.

In [121]:
%%file /tmp/unify.p

cnf(ax1, axiom, unifier(a(A),b(B)) | f(g(A)) != f(B)).
%cnf(ax1, axiom, g(A) != B  | f(g(A)) != f(B)).
% cnf(, ocnjecture, ?[A,B] : f(g(A)) = f(B)).  negate. ![A,B]: ~(f(g(A)) = f(B)).


Overwriting /tmp/unify.p


In [123]:
! eprover-ho --print-saturated --output-level=6 --literal-selection-strategy="SelectLargestNegLit" --proof-object /tmp/unify.p

cnf(ax1, axiom, (unifier(a(X1),b(X2))|f(g(X1))!=f(X2)), file('/tmp/unify.p', ax1)).
cnf(c_0_2, plain, (unifier(a(X1),b(X2))|f(g(X1))!=f(X2)),inference(fof_simplification, [status(thm)],[c_0_1])).
# Initializing proof state
cnf(c_0_3, plain, (unifier(a(X1),b(X2))|f(g(X1))!=f(X2)), c_0_-9223372036854775804,['eval']).
# Scanning for AC axioms
cnf(c_0_4, plain, (unifier(a(X1),b(X2))|f(g(X1))!=f(X2)), c_0_3,['new_given']).
cnf(c_0_5, plain, (unifier(a(X1),b(g(X1)))),inference(er,[status(thm)],[c_0_4])).
cnf(c_0_6, plain, (unifier(a(X1),b(g(X1)))), c_0_5,['eval']).
cnf(c_0_7, plain, (unifier(a(X1),b(g(X1)))), c_0_6,['new_given']).
cnf(c_0_8, plain, (unifier(a(X1),b(g(X1)))), c_0_7,['final']).
cnf(c_0_9, plain, (unifier(a(X1),b(X2))|f(g(X1))!=f(X2)), c_0_4,['final']).

# No proof found!
# SZS status Satisfiable
# SZS output start Saturation
cnf(ax1, axiom, (unifier(a(X1),b(X2))|f(g(X1))!=f(X2)), file('/tmp/unify.p', ax1)).
cnf(c_0_1, plain, (unifier(a(X1),b(X2))|f(g(X1))!=f(X2)), inference(fo

Also E-unification and lambda unification


In [59]:
%%file /tmp/eq.p
cnf(assoc, axiom, mul(X,mul(Y,Z)) = mul(mul(X,Y),Z)).
cnf(com, axiom, mul(Y,X) = mul(X,Y)).
%fof(mul_e, conjecture, ?[X,Y] : mul(mul(a,Y), X) = mul(X,mul(a,X))).
%fof(mul_e, conjecture, ?[X,Y,Z] : mul(mul(Z,Y), X) = mul(X,mul(Z,X))).


Overwriting /tmp/eq.p


In [60]:
!eprover-ho --print-saturated --output-level=6 --literal-selection-strategy="SelectLargestNegLit"  /tmp/eq.p

cnf(assoc, axiom, (mul(X1,mul(X2,X3))=mul(mul(X1,X2),X3)), file('/tmp/eq.p', assoc)).
cnf(com, axiom, (mul(X1,X2)=mul(X2,X1)), file('/tmp/eq.p', com)).
# Initializing proof state
cnf(c_0_3, plain, (mul(X1,X2)=mul(X2,X1)), c_0_-9223372036854775802,['eval']).
cnf(c_0_4, plain, (mul(mul(X1,X2),X3)=mul(X1,mul(X2,X3))), c_0_-9223372036854775803,['eval']).
# Scanning for AC axioms
# mul is AC
# AC handling enabled
cnf(c_0_5, plain, (mul(X1,X2)=mul(X2,X1)), c_0_3,['new_given']).
cnf(c_0_6, plain, (mul(mul(X1,X2),X3)=mul(X1,mul(X2,X3))), c_0_4,['new_given']).
cnf(c_0_7, plain, (mul(X1,mul(X2,X3))=mul(X3,mul(X1,X2))),inference(pm,[status(thm)],[c_0_5,c_0_6])).
cnf(c_0_8, plain, (mul(mul(X1,mul(X2,X3)),X4)=mul(mul(X1,X2),mul(X3,X4))),inference(pm,[status(thm)],[c_0_6,c_0_6])).
cnf(c_0_9, plain, (mul(X2,mul(X3,X1))=mul(X1,mul(X2,X3))),inference(pm,[status(thm)],[c_0_5,c_0_6])).
cnf(c_0_10, plain, (mul(mul(X2,X1),X3)=mul(X1,mul(X2,X3))),inference(pm,[status(thm)],[c_0_6,c_0_5])).
cnf(c_0_11, plain

In [58]:
! eprover-ho --answers=10 --term-ordering=KBO6 --literal-selection-strategy="SelectLargestNegLit" --conjectures-are-questions /tmp/eq.p

# Initializing proof state
# Scanning for AC axioms
# mul is AC
# AC handling enabled
#
#cnf(i_0_4, plain, (mul(X1,X2)=mul(X2,X1))).
#
#cnf(i_0_3, plain, (mul(mul(X1,X2),X3)=mul(X1,mul(X2,X3)))).
#
#cnf(i_0_6, plain, (mul(X1,mul(X2,X3))=mul(X3,mul(X1,X2)))).
##
#cnf(i_0_9, plain, (mul(X2,mul(X1,X3))=mul(X1,mul(X2,X3)))).
######
#cnf(i_0_26, plain, (mul(X3,mul(X2,X1))=mul(X1,mul(X2,X3)))).
################################
#cnf(i_0_5, negated_conjecture, ($answer(esk1_3(X3,X2,X1))|mul(X1,mul(X2,X3))!=mul(X3,mul(X1,X3)))).
## SZS status Theorem
# SZS answers Tuple [[X1, X1, X1]|_]

#cnf(i_0_103, negated_conjecture, ($answer(esk1_3(X1,X1,X1)))).
#
#cnf(i_0_111, negated_conjecture, ($answer(esk1_3(X1,X2,X3))|mul(X1,mul(X1,X3))!=mul(X3,mul(X2,X1)))).
##
#cnf(i_0_114, negated_conjecture, ($answer(esk1_3(X1,X2,X3))|mul(X3,mul(X1,X1))!=mul(X3,mul(X2,X1)))).
## SZS answers Tuple [[X1, X1, X2]|_]

#cnf(i_0_173, negated_conjecture, ($answer(esk1_3(X1,X1,X2)))).
#######
#cnf(i_0_127, negated_conjec

## Prolog


In [1]:
%%file /tmp/prolog.p

cnf(add_succ, axiom, add(s(X),Y,s(Z)) | ~add(X, Y, Z)).
cnf(add_z, axiom, add(z, Y, Y)).
fof(add_g, conjecture, ?[X,Y]: add(X,Y,s(s(z)))).


Writing /tmp/prolog.p


In [2]:
! eprover-ho --output-level=6 /tmp/prolog.p

cnf(add_succ, axiom, (add(s(X1),X2,s(X3))|~add(X1,X2,X3)), file('/tmp/prolog.p', add_succ)).
cnf(add_z, axiom, (add(z,X1,X1)), file('/tmp/prolog.p', add_z)).
fof(add_g, conjecture, ?[X2, X1]:(add(X2,X1,s(s(z)))), file('/tmp/prolog.p', add_g)).
fof(c_0_4, negated_conjecture, ~(?[X2, X1]:(add(X2,X1,s(s(z))))),inference(assume_negation, [status(cth)],[c_0_3])).
cnf(c_0_5, plain, (add(s(X1),X2,s(X3))|~add(X1,X2,X3)),inference(fof_simplification, [status(thm)],[c_0_1])).
fof(c_0_6, negated_conjecture, ![X2, X1]:(~add(X2,X1,s(s(z)))),inference(fof_nnf, [status(thm)],[c_0_4])).
fof(c_0_7, negated_conjecture, ![X4, X5]:(~add(X4,X5,s(s(z)))),inference(variable_rename, [status(thm)],[c_0_6])).
fof(c_0_8, negated_conjecture, ![X4, X5]:(~add(X4,X5,s(s(z)))),inference(fof_nnf, [status(thm)],[c_0_7])).
cnf(c_0_9, negated_conjecture, (~add(X1,X2,s(s(z)))),inference(split_conjunct, [status(thm)],[c_0_8])).
# Initializing proof state
cnf(c_0_10, plain, (add(z,X1,X1)), c_0_-9223372036854775801,['eval'])

In [94]:
! eprover-ho --conjectures-are-questions  --answers=3 /tmp/prolog.p

# Initializing proof state
# Scanning for AC axioms
#
#cnf(i_0_4, plain, (add(z,X1,X1))).
#
#cnf(i_0_3, plain, (add(s(X1),X2,s(X3))|~add(X1,X2,X3))).
#
#cnf(i_0_5, negated_conjecture, ($answer(esk1_2(X1,X2))|~add(X1,X2,s(s(z))))).
## SZS status Theorem
# SZS answers Tuple [[z, s(s(z))]|_]

#cnf(i_0_6, negated_conjecture, ($answer(esk1_2(z,s(s(z)))))).
#
#cnf(i_0_7, negated_conjecture, ($answer(esk1_2(s(X1),X2))|~add(X1,X2,s(z)))).
## SZS answers Tuple [[s(z), s(z)]|_]

#cnf(i_0_8, negated_conjecture, ($answer(esk1_2(s(z),s(z))))).
#
#cnf(i_0_9, negated_conjecture, ($answer(esk1_2(s(s(X1)),X2))|~add(X1,X2,z))).
## SZS answers Tuple [[s(s(z)), z]|_]

# Proof found!


## Simplifier





In [69]:
%%file /tmp/simp.p
cnf(mul_zero, axiom, mul(X,zero) = zero).
cnf(mul_comm, axiom, mul(X,Y) = mul(Y,X)).
cnf(mul_assoc, axiom, mul(X,mul(Y,Z)) = mul(mul(X,Y),Z)).
cnf(add_comm, axiom, add(X,Y) = add(Y,X)).
cnf(add_assoc, axiom, add(X,add(Y,Z)) = add(add(X,Y),Z)).
cnf(add_zero, axiom, add(X,zero) = X).
cnf(abc, axiom, add(a,b) = c).

cnf(goal_expr, axiom, t = add(add(a, mul(a,zero)), add(zero, b))).


Overwriting /tmp/simp.p


 --processed-clauses-limit=100 

In [ ]:
! eprover-ho --print-saturated --unprocessed-limit=100 --output-level=6 /tmp/simp.p

## Functional programs.

Constructors should come less in the precendence.
LPO is more like functional programming.
KBO is more like simplification


Without the precendence annotations, eprover is not terminating

In [ ]:
%%file /tmp/add.p

cnf(add_succ, axiom, add(s(X),Y) = s(add(X,Y))).
cnf(add_z, axiom, add(z, Y) = Y).
cnf(addn, axiom, add())

Writing /tmp/add.p


In [ ]:
! eprover-ho --print-saturated  /tmp/add.p % non terminating

auto is not better?

In [101]:
! eprover-ho --print-saturated --auto-schedule /tmp/add.p

# Preprocessing class: FSSSSMSSSSSNFFN.
# Scheduled 1 strats onto 1 cores with 300 seconds (300 total)
# Starting G-E--_302_C18_F1_URBAN_RG_S04BN with 300s (1) cores
^C


In [98]:
! eprover-ho --print-saturated --term-ordering=LPO4 --precedence="add>s>z" /tmp/add.p

# Initializing proof state
# Scanning for AC axioms
#
#cnf(i_0_4, plain, (add(z,X1)=X1)).
#
#cnf(i_0_3, plain, (add(s(X1),X2)=s(add(X1,X2)))).

# No proof found!
# SZS status Satisfiable
# Processed positive unit clauses:
cnf(i_0_4, plain, (add(z,X1)=X1)).
cnf(i_0_3, plain, (add(s(X1),X2)=s(add(X1,X2)))).

# Processed negative unit clauses:

# Processed non-unit clauses:

# Unprocessed positive unit clauses:

# Unprocessed negative unit clauses:

# Unprocessed non-unit clauses:




Older notes
E prover
enormalizer is an interesting sounding program. Give it a pile of unit equalities and it will normalize with respect to thm EPR grounder to DIMACS The e calculus is a bit puzzling. I haven’t seen the analog for vampire

2.6 manual It’s also in the github repo if you make doc

I like how –answer mode works a little better for e.

Database printing feature -S. Doesn’t print stuff I would expect though? Kind of prints everything by default right? Early stopping conditions clause size

eprover --help | less
--watchlist - “constructive” proofs that aren’t seeded by refuation training - you can train it if you have a database of indicative theorems. This might be useful if you have a sequence of increasingly hard theorems, or if you are making a tool that spits out formula. -S print saturated clause set -W literaeelection stretagory NoGeneration will inhibit all generating instances “Each of the strategies that do actually select negative literals has a corresponding counterpart starting with P that additionally allows paramodulation into maximal positive literals”

echo "
fof(ground, axiom,
    edge(a,b) & edge(b,c)
).
fof(path1, axiom,
    ![X,Y]: (edge(X,Y) => path(X,Y))
).
fof(path2, axiom,
    ![X,Y,Z]:  ((edge(X,Y) & path(Y,Z)) => path(X,Z))
).
" | eprover  --literal-selection-strategy=SelectNegativeLiterals
--generated-limit=100 Ok this basically did what i wanted. I’m not sure what it is though?

“The most natural clause representation for E is probably a literal disjunction: a=
true;c!=$true.”


## Equality Saturation
Somewhat like datalog, but it's worse here. The ordering is really important to get fair search.

We want the goal term in immediately.


%%file /tmp/eq.p
cnf(goalterm, axiom, t = div(mul(a, two), two)).
cnf(mulcomm, axiom, mul(X,Y) = mul(Y,X)).
cnf(mulassoc, axiom, mul(X,mul(Y,Z)) = mul(mul(X,Y),Z)).
cnf(mulunit, axiom, mul(X, one) = X).
cnf(mulinv, axiom, div(mul(X,Y), Y) = X | Y = zero).
cnf(shiftmul, axiom, mul(X, two) = shift(X, one)).

# Vampire

Maybe I should play with these.
--memory
--cores

--selection (-s)
        Selection methods 2,3,4,10,11 are complete by virtue of extending Maximal 
        i.e. they select the best among maximal. Methods 1002,1003,1004,1010,1011 
        relax this restriction and are therefore not complete.
         0     - Total (select everything)
         1     - Maximal
         2     - ColoredFirst, MaximalSize then Lexicographical
         3     - ColoredFirst, NoPositiveEquality, LeastTopLevelVariables,
                  LeastDistinctVariables then Lexicographical
         4     - ColoredFirst, NoPositiveEquality, LeastTopLevelVariables,
                  LeastVariables, MaximalSize then Lexicographical
         10    - ColoredFirst, NegativeEquality, MaximalSize, Negative then Lexicographical
         11    - Lookahead
         666   - Random
         1002  - Incomplete version of 2
         1003  - Incomplete version of 3
         1004  - Incomplete version of 4
         1010  - Incomplete version of 10
         1011  - Incomplete version of 11
         1666  - Incomplete version of 666
        Or negated, which means that reversePolarity is true i.e. for selection 
        we treat all negative non-equality literals as positive and vice versa 
        (can only apply to non-equality literals).
        
        default: 10

--symbol_precedence (-sp)
        Vampire uses term orderings which require a precedence relation between 
        symbols.
        Arity orders symbols by their arity (and reverse_arity takes the reverse 
        of this) and occurrence orders symbols by the order they appear in the 
        problem. Then we have a few precedence generating schemes adopted from 
        E: frequency - sort by frequency making rare symbols large, reverse does 
        the opposite, (For the weighted versions, each symbol occurrence counts 
        as many times as is the length of the clause in which it occurs.) unary_first 
        is like arity, except that unary symbols are maximal (and ties are broken 
        by frequency), unary_frequency is like frequency, except that unary symbols 
        are maximal, const_max makes constants the largest, then falls back to 
        arity, const_min makes constants the smallest, then falls back to reverse_arity, 
        const_frequency makes constants the smallest, then falls back to frequency.
        default: arity
        values: arity,occurrence,reverse_arity,unary_first,const_max,const_min,scramble,
                frequency,unary_frequency,const_frequency,reverse_frequency,
                weighted_frequency,reverse_weighted_frequency
--term-rdering kbo lpo
--induction = none, stuct, int,both

default on rules

--backward_demodulation (-bd)
        Oriented rewriting of kept clauses by newly derived unit equalities
        s = t     L[sθ] \/ C
        ---------------------   where sθ > tθ (replaces RHS)
         L[tθ] \/ C
        
        default: all
        values: all,off,preordered
--binary_resolution (-br)
        Standard binary resolution i.e.
        C \/ t     D \/ s
        ---------------------
        (C \/ D)θ
        where θ = mgu(t,-s) and t selected
        default: on
--demodulation_redundancy_check (-drc)
        The following cases of backward and forward demodulation do not preserve 
        completeness:
        s = t     s = t1 \/ C    s = t     s != t1 \/ C
        ---------------------    ---------------------
        t = t1 \/ C              t != t1 \/ C
        where t > t1 and s = t > C (RHS replaced)
        With `on`, we check this condition and don't demodulate if we could violate 
        completeness.
        With `encompass`, we treat demodulations (both forward and backward) as 
        encompassment demodulations (as defined by Duarte and Korovin in 2022's 
        IJCAR paper).
        With `off`, we skip the checks, save time, but become incomplete.
        default: on
        values: off,encompass,on
    --forward_demodulation (-fd)
        Oriented rewriting of newly derived clauses by kept unit equalities
        s = t     L[sθ] \/ C
        ---------------------  where sθ > tθ
         L[tθ] \/ C
        If 'preordered' is set, only equalities s = t where s > t are used for 
        rewriting.
        default: all
        values: all,off,preordered
--forward_subsumption (-fs)
        Perform forward subsumption deletion.
        default: on
--forward_subsumption_resolution (-fsr)
        Perform forward subsumption resolution.
        default: on
--simultaneous_superposition (-sims)
        Rewrite the whole RHS clause during superposition, not just the target 
        literal.
        default: on
--superposition (-sup)
        Control superposition. Turning off this core inference leads to an incomplete 
        calculus on equational problems.
        default: on
--superposition_from_variables (-sfv)
        Perform superposition from variables.
        default: on

--unit_resulting_resolution (-urr)
        Uses unit resulting resolution only to derive empty clauses (may be useful 
        for splitting). 'ec_only' only derives empty clauses, 'on' does everything 
        (but implements a heuristic to skip deriving more than one empty clause), 
        'full' ignores this heuristic and is thus complete also under AVATAR.
        default: off
        values: ec_only,off,on,full

--show_ordering
--show_induction
--manual_cs clause select manually
--show_everything


Pretty interesting

In [ ]:
! vampire --show_everything on --manual_cs on /tmp/path.p